In [ ]:
import numpy
import bempp.api
from bempp.api.operators.boundary import sparse, laplace
from scipy.sparse.linalg import gmres

In [ ]:
def rhs_fun(x, n, domain_index,result):
    global phi_static
    result[:] = phi_static

def energy_sphere(R, d_stern, vertices, elements):
    R_diel = R - d_stern

    grid_diel = bempp.api.grid_from_element_data(R_diel*vertices, elements)
    grid_stern = bempp.api.grid_from_element_data((R_diel+d_stern)*vertices, elements)

    space_diel = bempp.api.function_space(grid_diel, "DP", 0)
    space_stern = bempp.api.function_space(grid_stern, "DP", 0)
    
    N_diel = grid_diel.leaf_view.entity_count(0)
    N_stern = grid_stern.leaf_view.entity_count(0)

    phis_grid_fun = bempp.api.GridFunction(space_diel, fun=rhs_fun)

    rhs = numpy.concatenate([phis_grid_fun.coefficients, 
                      numpy.zeros(N_stern)])
    
    M11   = laplace.single_layer(space_diel, space_diel, space_diel)
    M12   = laplace.single_layer(space_stern, space_diel, space_diel)
    M21   = laplace.single_layer(space_diel, space_stern, space_stern)
    M22   = laplace.single_layer(space_stern, space_stern, space_stern)

    blocked = bempp.api.BlockedOperator(2, 2)
    blocked[0,0] = M11
    blocked[0,1] = M12
    blocked[1,0] = M21
    blocked[1,1] = M22
    op_discrete = blocked.strong_form()
    
    sigma, info = gmres(op_discrete, rhs, callback=iteration_counter, tol=1e-5, maxiter=500, restart = 1000)
    
    sigma_d = sigma[:N_diel]
    sigma_s = sigma[N_diel:]
    
    elements_d = list(grid_diel.leaf_view.entity_iterator(0))
    elements_s = list(grid_stern.leaf_view.entity_iterator(0))

    area_d = numpy.zeros(N_diel)
    area_s = numpy.zeros(N_stern)

    for i in range(N_diel):
        area_d[i] = elements_d[i].geometry.volume
    for i in range(N_stern):
        area_s[i] = elements_s[i].geometry.volume
        
    energy = 0.5*numpy.sum(area_d*sigma_d*phi_static)
    
    e_a = energy/(4*numpy.pi*(R_diel+d_stern)**2)
    
    #e_a = energy/(4*numpy.pi*(R_diel)**2)
    return energy, e_a

In [ ]:
grid_base = bempp.api.shapes.sphere(h=0.1)

vertices = grid_base.leaf_view.vertices 
elements = grid_base.leaf_view.elements 

d_stern = 1.4
phi_static = 12.

Radius = 10*numpy.array([0.15903614457831333, 
0.23614457831325308, 
0.35180722891566263, 
0.48032128514056227, 
0.6409638554216868, 
0.8016064257028112, 
0.9590361445783132, 
1.1293172690763051, 
1.3542168674698791, 
1.6112449799196784, 
1.8811244979919675])

In [ ]:
energy_area = numpy.zeros(len(Radius))
energy      = numpy.zeros(len(Radius))

for i in range(len(energy)):
    energy[i], energy_area[i] = energy_sphere(Radius[i], d_stern, vertices, elements)

In [ ]:
from matplotlib import pyplot

E_area_chandler = numpy.array([12.995391705069125,
24.239631336405523,
33.64055299539171,
44.51612903225807,
51.1520737327189,
55.20737327188941,
57.78801843317973,
59.44700460829494,
61.65898617511521,
64.0552995391705,
64.60829493087559])

In [ ]:
pyplot.plot(Radius, energy_area, marker='+', ls='', label='BEM++')
pyplot.plot(Radius, E_area_chandler, marker='o', ls='', label='Chandler')
pyplot.legend(loc='best')

In [ ]:
pyplot.savefig('nonpolar_energy.pdf')

In [ ]:
E_chandler = E_area_chandler*4*numpy.pi*Radius**2
pyplot.plot(Radius, energy, marker='+', ls='', label='BEM++')
pyplot.plot(Radius, E_chandler, marker='o', ls='', label='Chandler')
pyplot.legend(loc='best')